<a href="https://colab.research.google.com/github/ipeirotis/autoencoders_census/blob/main/autoencoder_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np


In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.models import Model

def create_autoencoder(input_dim, latent_dim=1, hidden_nodes=16):
    """
    Create an autoencoder model.

    Parameters:
    input_dim (int): The dimension of the input data.
    latent_dim (int): The dimension of the latent space. Default is 1.
    hidden_nodes (int): The number of nodes in the hidden layer. Default is 16.

    Returns:
    encoder (Model): The encoder part of the autoencoder.
    decoder (Model): The decoder part of the autoencoder.
    autoencoder (Model): The whole autoencoder.
    """

    # ENCODER PART
    # Define the input to the encoder
    input_encoder = Input(shape=(input_dim,), name="Encoder_Input")

    # Apply batch normalization
    encoder_batch_norm1 = BatchNormalization()(input_encoder)

    # Hidden layer of the encoder
    encoder_hidden = Dense(hidden_nodes, activation="relu", name="Encoder_Hidden")(encoder_batch_norm1)

    # Apply another batch normalization
    encoder_batch_norm2 = BatchNormalization()(encoder_hidden)

    # Output of the encoder
    z = Dense(latent_dim, name="Encoder_Output")(encoder_batch_norm2)

    # Define the whole encoder model
    encoder = Model(input_encoder, z, name="Encoder")

    # DECODER PART
    # Define the input to the decoder
    input_decoder = Input(shape=(latent_dim,), name="Decoder_Input")

    # Apply batch normalization
    decoder_batch_norm1 = BatchNormalization()(input_decoder)

    # Hidden layer of the decoder
    decoder_hidden = Dense(hidden_nodes, activation="relu", name="Decoder_Hidden")(decoder_batch_norm1)

    # Apply another batch normalization
    decoder_batch_norm2 = BatchNormalization()(decoder_hidden)

    # Output of the decoder
    decoded = Dense(input_dim, activation="linear", name="Decoder_Output")(decoder_batch_norm2)

    # Define the whole decoder model
    decoder = Model(input_decoder, decoded, name="Decoder")

    # AUTOENCODER
    # Pass the input through the encoder and the decoder
    encoder_decoder = decoder(encoder(input_encoder))

    # Define the whole autoencoder model
    autoencoder = Model(input_encoder, encoder_decoder)

    return encoder, decoder, autoencoder


In [ ]:

def train_model(autoencoder, transformed_df, epochs=10, batch_size=20, validation_split=0.2):
    def masked_mse(y_true, y_pred):
        mask = tf.where(tf.math.is_nan(y_true), 0.0, 1.0)
        return tf.reduce_mean(tf.square(y_true - y_pred) * mask)

    autoencoder.compile(loss=masked_mse, optimizer="adam")

    mask = np.where(transformed_df.isnull(), 0.0, 1.0)
    mask = np.expand_dims(mask, axis=-1)

    transformed_df = transformed_df.fillna(0.0)

    # Initialize lists to track losses
    train_loss = []
    val_loss = []

    class LossTracker(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            train_loss.append(logs.get('loss'))
            val_loss.append(logs.get('val_loss'))

    loss_tracker = LossTracker()

    history = autoencoder.fit(
        transformed_df, transformed_df, sample_weight=mask, shuffle=True, epochs=epochs, batch_size=batch_size,
        validation_split=validation_split, verbose=0, callbacks=[loss_tracker]
    )

    return history, train_loss, val_loss
